# Validating Responses using Regex

This notebook demonstrates how to synthesize regex-validated responses from an LLM. 

In this example, we'll ask GPT-3.5 to give us a phone number, and make sure it's just separated by dashes - so something of the format 123-456-7890.


### Define the RegEx

In [1]:
regex = '\\d{3}-\\d{3}-\\d{4}'

### Import Guardrails and openai

In [2]:
import openai
from guardrails import Guard
from guardrails.validators import RegexMatch
from rich import print

In [3]:
#Otel setup

from opentelemetry.sdk.resources import SERVICE_NAME, Resource

from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

from opentelemetry import metrics
from opentelemetry.exporter.otlp.proto.grpc.metric_exporter import OTLPMetricExporter
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader

# Service name is required for most backends
resource = Resource(attributes={
    SERVICE_NAME: "gr"
})

traceProvider = TracerProvider(resource=resource)
processor = BatchSpanProcessor(OTLPSpanExporter(endpoint="0.0.0.0:4317", insecure=True))
traceProvider.add_span_processor(processor)
trace.set_tracer_provider(traceProvider)

reader = PeriodicExportingMetricReader(
    OTLPMetricExporter(endpoint="0.0.0.0:4317") # old port was 5555
)
meterProvider = MeterProvider(resource=resource, metric_readers=[reader])
metrics.set_meter_provider(meterProvider)

tracer = trace.get_tracer("gr")


In [4]:
# tracer
# from opentelemetry import trace
# from opentelemetry.sdk.trace import TracerProvider
# # from opentelemetry.ext.otcollector.trace_exporter  import CollectorSpanExporter
# from opentelemetry.sdk.trace.export import (
#     BatchSpanProcessor,
#     ConsoleSpanExporter,
# )

# provider = TracerProvider()
# processor = BatchSpanProcessor(ConsoleSpanExporter())
# provider.add_span_processor(processor)

# # Sets the global default tracer provider
# trace.set_tracer_provider(provider)

# # Creates a tracer from the global tracer provider
# tracer = trace.get_tracer("gr")


### Create the guard and prompt

We use a simple string guard with the RegexMatch validator. We can see that the initial response is malformatted, but the reask gets the LLM to respond correctly.

In [5]:
guard = Guard.from_string(
    validators=[RegexMatch(regex=regex, match_type='search', on_fail='reask')],
    tracer=tracer
)

# guard(
#     prompt='I am writing a movie and need a fake phone number. Generate a fake phone number. Do not write anything other than the phone number.',
#     llm_api=openai.chat.completions.create,
# )

guard.parse("Sure! Here's a fake phone number for your movie: 555-123-4567.")

print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ No prompt                                                                                               │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Sure! Here's a fake phone number for your movie: 555-123-4567.                                          │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ "Sure! Here's a fake phone number for your movie: 555-123-4567."                                        │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯